<a href="https://colab.research.google.com/github/Ryandonofrio3/2-Way-Tracker/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.817600
2,2.304200
3,1.689300
4,1.938200
5,1.656900
6,1.621900
7,1.187100
8,1.264200
9,1.101200
10,1.189500


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

462.7198 seconds used for training.
7.71 minutes used for training.
Peak reserved memory = 7.922 GB.
Peak reserved memory for training = 1.938 GB.
Peak reserved memory % of max memory = 53.716 %.
Peak reserved memory for training % of max memory = 13.141 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [6]:
# -*- coding: utf-8 -*-
"""
PokerBench Fine-Tuning with Llama 3.1 8B (Phase 1)
This notebook fine-tunes Llama 3.1 8B using the PokerBench dataset,
which has fields "instruction" (the poker scenario) and "output" (the optimal action).
It evaluates the base model’s performance (action accuracy), runs a small supervised fine-tune (Phase 1),
and then re-evaluates the model.
"""

# ----- Installation -----
# Uncomment the following lines if you need to install Unsloth in Colab.
# !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
# !pip install --no-deps cut_cross_entropy unsloth_zoo
# !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
# !pip install --no-deps unsloth

# ----- Imports & Setup -----
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset

max_seq_length = 2048  # Adjust as needed
dtype = None           # Auto-detect dtype (use Float16 for Tesla T4, BF16 for Ampere+)
load_in_4bit = True    # Use 4-bit quantization to reduce memory usage

# ----- Load the Base Model -----
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Add LoRA adapters to update only a small fraction of parameters.
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Suggested values: 8, 16, 32, etc.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Optimized for long contexts
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

EOS_TOKEN = tokenizer.eos_token

# ----- Data Preparation: PokerBench -----
# Load the PokerBench dataset. In this version, the keys are "instruction" and "output".
raw_train_dataset = load_dataset("RZ412/PokerBench", split="train")
raw_val_dataset = load_dataset("RZ412/PokerBench", split="test")

# Print column names to verify.
print("Train dataset columns:", raw_train_dataset.column_names)

# Define a prompt template for supervised training.
# Here we use the instruction and then append "Your optimal action is:" followed by the output.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instr, out in zip(instructions, outputs):
        # For training, we include the expected answer.
        text = f"{instr}\nYour optimal action is: {out}{EOS_TOKEN}"
        texts.append(text)
    return {"text": texts}

# Apply formatting to the training dataset.
train_dataset = raw_train_dataset.map(formatting_prompts_func, batched=True)

# Define an evaluation prompt function.
# For evaluation, we present the instruction and ask for "Your optimal action is:" without providing the answer.
def evaluation_prompt(instruction):
    return f"{instruction}\nYour optimal action is:"

# ----- Inference Preparation -----
# IMPORTANT: Call for_inference before any inference.
FastLanguageModel.for_inference(model)

# ----- Evaluation Function -----
def evaluate_action_accuracy(model, tokenizer, raw_dataset, num_examples=50):
    model.eval()
    correct = 0
    total = 0
    for i in range(min(num_examples, len(raw_dataset))):
        example = raw_dataset[i]
        instr = example["instruction"]
        true_action = example["output"].strip()
        prompt = evaluation_prompt(instr)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=64)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract the generated action: take text after "Your optimal action is:"
        if "Your optimal action is:" in generated:
            pred_action = generated.split("Your optimal action is:")[-1].strip().split("\n")[0].strip()
        else:
            pred_action = generated.strip().split("\n")[0].strip()
        if pred_action == true_action:
            correct += 1
        total += 1
    accuracy = correct / total if total > 0 else 0
    print(f"Action Accuracy (Exact Match) on {total} examples: {accuracy*100:.2f}%")
    return accuracy

# ----- Baseline Evaluation -----
print("=== Evaluating Base (Pre-Fine-Tuned) Model on PokerBench Validation ===")
evaluate_action_accuracy(model, tokenizer, raw_val_dataset, num_examples=50)

# ----- Fine-Tuning Setup using TRL's SFTTrainer -----
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Set True to speed up training with shorter sequences
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # Start with a small number of steps for baseline testing.
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Change to "wandb" if desired.
    ),
)

# Display GPU memory info before training.
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved at start.")

# ----- Fine-Tune the Model -----
trainer_stats = trainer.train()

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"Training time: {trainer_stats.metrics['train_runtime']} seconds.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# ----- Evaluation after Fine-Tuning -----
print("=== Evaluating Fine-Tuned Model on PokerBench Validation ===")

model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

FastLanguageModel.for_inference(model)

model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

evaluate_action_accuracy(model, tokenizer, raw_val_dataset, num_examples=50)

# ----- Inference Example -----
# Generate a response for a sample poker scenario.
sample = raw_val_dataset[0]
instr = sample["instruction"]
prompt = evaluation_prompt(instr)
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=64)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
predicted_action = generated.split("Your optimal action is:")[-1].strip().split("\n")[0].strip()

print("\n=== Inference Example ===")
print("Sample Instruction:")
print(instr)
print("True Optimal Action:", sample["output"])
print("Model Generated Action:", predicted_action)

# ----- Saving the LoRA Adapters -----



==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Generating train split:   0%|          | 0/563200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11000 [00:00<?, ? examples/s]

Using the latest cached version of the dataset since RZ412/PokerBench couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/RZ412___poker_bench/default/0.0.0/9515e6bbed93add724a9aa994e0b1ce66e71a8b2 (last modified on Sun Mar  2 21:37:11 2025).


...

Truncating train dataset (num_proc=2):   0%|          | 0/563200 [00:00<?, ? examples/s]

GPU = Tesla T4. Max memory = 14.741 GB.
6.203 GB of memory reserved at start.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 563,200 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.084000
2,2.097300
3,2.046200
4,1.985300
5,1.752500
6,1.512800
7,1.286300
8,0.917800
9,0.679300
10,0.500800


Training time: 468.0333 seconds.
Peak reserved memory = 6.717 GB.
Peak reserved memory for training = 0.514 GB.
Peak reserved memory % of max memory = 45.567 %.
Peak reserved memory for training % of max memory = 3.487 %.
=== Evaluating Fine-Tuned Model on PokerBench Validation ===
Action Accuracy (Exact Match) on 50 examples: 38.00%

=== Inference Example ===
Sample Instruction:


You are a specialist in playing 6-handed No Limit Texas Holdem. The following will be a game scenario and you need to make the optimal decision.

Here is a game summary:

The small blind is 0.5 chips and the big blind is 1 chips. Everyone started with 100 chips.
The player positions involved in this game are UTG, HJ, CO, BTN, SB, BB.
In this hand, your position is BTN, and your holding is [King of Heart and Three of Heart].
Before the flop, BTN raise 2.5 chips, and BB call. Assume that all other players that is not mentioned folded.
The flop comes Ten Of Heart, Three Of Spade, and Two Of Diamond, then BB bet

In [ ]:
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# Load the dataset
ds = load_dataset("RZ412/PokerBench")
print("Dataset loaded:", ds)

# Format the dataset into a single 'text' field
def format_example(example):
    return {
        "text": f"### Instruction: {example['instruction']}\n### Response: {example['output']} <|eos|>"
    }

train_dataset = ds["train"].map(format_example, remove_columns=ds["train"].column_names)
test_dataset = ds["test"].map(format_example, remove_columns=ds["test"].column_names)

# Load the model and tokenizer (with LoRA from checkpoint)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./llama-3.1-8b-pokerbench-lora",  # Load from your last save
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True
)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42
)

# Training arguments for scaling to 600 steps
training_args = TrainingArguments(
    output_dir="./llama-3.1-8b-pokerbench",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Total batch size = 8
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=600,  # 10x original 60 steps
    fp16=True,
    logging_steps=100,
    save_steps=500,
    optim="adamw_8bit",
    report_to="none"
)

# Set up trainer and resume training
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args
)
print("Resuming training from checkpoint...")
trainer.train(resume_from_checkpoint="./llama-3.1-8b-pokerbench/checkpoint-60")

# Save the scaled-up model
model.save_pretrained("./llama-3.1-8b-pokerbench-lora-600")
tokenizer.save_pretrained("./llama-3.1-8b-pokerbench-lora-600")
print("Model saved at './llama-3.1-8b-pokerbench-lora-600'")

# Evaluation function for AA and EM
def evaluate_model(model, tokenizer, test_dataset, num_examples=50):
    model.eval()
    aa_correct = 0  # Action Accuracy
    em_correct = 0  # Exact Match
    total = min(num_examples, len(test_dataset))

    for i in range(total):
        example = test_dataset[i]
        instruction = example["text"].split("### Response:")[0] + "### Response:"
        true_output = example["text"].split("### Response:")[1].strip().replace(" <|eos|>", "")

        # Generate model prediction
        inputs = tokenizer(instruction, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
        pred_output = tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Response:")[1].strip()

        # Extract action (first word) and full prediction
        true_action = true_output.split()[0].lower()
        pred_action = pred_output.split()[0].lower()

        # Action Accuracy: Check if action type matches
        if true_action == pred_action:
            aa_correct += 1
            # Exact Match: Check if entire output matches (including bet size if present)
            if true_output == pred_output:
                em_correct += 1

    aa_score = (aa_correct / total) * 100
    em_score = (em_correct / total) * 100
    return aa_score, em_score

# Evaluate on test set (first 50 examples for speed, adjust as needed)
print("=== Evaluating Fine-Tuned Model on PokerBench Test Set ===")
# ----- Inference Preparation -----
# IMPORTANT: Call for_inference before any inference.
FastLanguageModel.for_inference(model)

aa, em = evaluate_model(model, tokenizer, test_dataset, num_examples=50)
print(f"Action Accuracy (AA) on 50 examples: {aa:.2f}%")
print(f"Exact Match (EM) on 50 examples: {em:.2f}%")

# Inference example
sample_instruction = """
You are a specialist in playing 6-handed No Limit Texas Holdem. The following will be a game scenario and you need to make the optimal decision.

Here is a game summary:

The small blind is 0.5 chips and the big blind is 1 chips. Everyone started with 100 chips.
The player positions involved in this game are UTG, HJ, CO, BTN, SB, BB.
In this hand, your position is BTN, and your holding is [King of Heart and Three of Heart].
Before the flop, BTN raise 2.5 chips, and BB call. Assume that all other players that is not mentioned folded.
The flop comes Ten Of Heart, Three Of Spade, and Two Of Diamond, then BB bet 4 chips, and BTN call.
The turn comes Five Of Diamond, then BB check.

Now it is your turn to make a move.
To remind you, the current pot size is 13.0 chips, and your holding is [King of Heart and Three of Heart].

Decide on an action based on the strength of your hand on this board, your position, and actions before you. Do not explain your answer.
Your optimal action is:
"""
true_action = "check"

# Generate prediction
inputs = tokenizer(f"### Instruction: {sample_instruction}\n### Response:", return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
pred_output = tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Response:")[1].strip()

print("=== Inference Example ===")
print(f"True Optimal Action: {true_action}")
print(f"Model Generated Action: {pred_output}")

In [4]:
# ----- Reload Fine-Tuned Model and Evaluate on PokerBench -----
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Reload the fine-tuned model and tokenizer from your saved directory "lora_model"
model, tokenizer = FastLanguageModel.from_pretrained(
    "lora_model",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# IMPORTANT: Prepare the reloaded model for inference.
FastLanguageModel.for_inference(model)

# Reload the PokerBench validation dataset
raw_val_dataset = load_dataset("RZ412/PokerBench", split="test")

# Define an evaluation prompt function.
def evaluation_prompt(instruction):
    return f"{instruction}\nYour optimal action is:"

# Define the evaluation function.
def evaluate_action_accuracy(model, tokenizer, raw_dataset, num_examples=50):
    model.eval()
    correct = 0
    total = 0
    for i in range(min(num_examples, len(raw_dataset))):
        example = raw_dataset[i]
        instr = example["instruction"]
        true_action = example["output"].strip()
        prompt = evaluation_prompt(instr)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=64)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract the generated action after the prompt.
        if "Your optimal action is:" in generated:
            pred_action = generated.split("Your optimal action is:")[-1].strip().split("\n")[0].strip()
        else:
            pred_action = generated.strip().split("\n")[0].strip()
        if pred_action == true_action:
            correct += 1
        total += 1
    accuracy = correct / total if total > 0 else 0
    print(f"Action Accuracy (Exact Match) on {total} examples: {accuracy*100:.2f}%")
    return accuracy

# Run the evaluation
accuracy = evaluate_action_accuracy(model, tokenizer, raw_val_dataset, num_examples=50)
print(f"Reloaded model evaluation accuracy: {accuracy*100:.2f}%")


FileNotFoundError: lora_model/*.json (invalid repository id)

In [ ]:
# -*- coding: utf-8 -*-
"""
PokerBench Fine-Tuning with Llama 3.1 8B (Phase 1 Scaling)
Resumes from checkpoint-60, scales to 600 steps, and evaluates AA and EM.
"""

# ----- Imports & Setup -----
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None  # Auto-detect (Float16 for Tesla T4)
load_in_4bit = True  # 4-bit quantization

# ----- Load the Dataset -----
ds = load_dataset("RZ412/PokerBench")
print("Dataset loaded:", ds)
raw_train_dataset = ds["train"]
raw_val_dataset = ds["test"]
print("Train dataset columns:", raw_train_dataset.column_names)

# ----- Data Formatting -----
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instr, out in zip(instructions, outputs):
        text = f"{instr}\nYour optimal action is: {out}{tokenizer.eos_token}"
    return {"text": texts}

train_dataset = raw_train_dataset.map(formatting_prompts_func, batched=True)

# ----- Load Model and Tokenizer -----
# Load from "lora_model" if that's your initial save; otherwise, adjust to correct path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="lora_model" if torch.cuda.is_available() else "unsloth/Meta-Llama-3.1-8B",  # Use local if exists
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Add LoRA adapters (re-apply if loading base model)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

EOS_TOKEN = tokenizer.eos_token

# ----- Training Setup -----
training_args = TrainingArguments(
    output_dir="./llama-3.1-8b-pokerbench",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Total batch size = 8
    warmup_steps=5,
    max_steps=600,  # Scale to 600 steps
    learning_rate=2e-4,  # From your first code
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=100,
    save_steps=500,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

# ----- Resume Training -----
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved at start.")
print("Resuming training from checkpoint...")
trainer_stats = trainer.train(resume_from_checkpoint="./llama-3.1-8b-pokerbench/checkpoint-60")

# Memory stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
print(f"Training time: {trainer_stats.metrics['train_runtime']} seconds.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")

# ----- Save Scaled Model -----
model.save_pretrained("./llama-3.1-8b-pokerbench-lora-600")
tokenizer.save_pretrained("./llama-3.1-8b-pokerbench-lora-600")
print("Model saved at './llama-3.1-8b-pokerbench-lora-600'")

# ----- Evaluation Function for AA and EM -----
def evaluate_model(model, tokenizer, raw_dataset, num_examples=50):
    FastLanguageModel.for_inference(model)
    model.eval()
    aa_correct = 0  # Action Accuracy
    em_correct = 0  # Exact Match
    total = min(num_examples, len(raw_dataset))

    for i in range(total):
        example = raw_dataset[i]
        instr = example["instruction"]
        true_output = example["output"].strip()
        prompt = f"{instr}\nYour optimal action is:"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
        pred_output = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Your optimal action is:")[-1].strip()

        # Extract action type
        true_action = true_output.split()[0].lower()
        pred_action = pred_output.split()[0].lower() if pred_output else ""

        # AA: Check action type
        if true_action == pred_action:
            aa_correct += 1
            # EM: Check full match (including bet size if present)
            if true_output == pred_output:
                em_correct += 1

    aa_score = (aa_correct / total) * 100
    em_score = (em_correct / total) * 100
    return aa_score, em_score

# ----- Evaluate Fine-Tuned Model -----
print("=== Evaluating Fine-Tuned Model on PokerBench Test Set ===")
# IMPORTANT: Prepare the reloaded model for inference.
FastLanguageModel.for_inference(model)

aa, em = evaluate_model(model, tokenizer, raw_val_dataset, num_examples=50)
print(f"Action Accuracy (AA) on 50 examples: {aa:.2f}%")
print(f"Exact Match (EM) on 50 examples: {em:.2f}%")

# ----- Inference Example -----
sample_instruction = """
You are a specialist in playing 6-handed No Limit Texas Holdem. The following will be a game scenario and you need to make the optimal decision.

Here is a game summary:
The small blind is 0.5 chips and the big blind is 1 chips. Everyone started with 100 chips.
The player positions involved in this game are UTG, HJ, CO, BTN, SB, BB.
In this hand, your position is BTN, and your holding is [King of Heart and Three of Heart].
Before the flop, BTN raise 2.5 chips, and BB call. Assume that all other players that is not mentioned folded.
The flop comes Ten Of Heart, Three Of Spade, and Two Of Diamond, then BB bet 4 chips, and BTN call.
The turn comes Five Of Diamond, then BB check.

Now it is your turn to make a move.
To remind you, the current pot size is 13.0 chips, and your holding is [King of Heart and Three of Heart].

Decide on an action based on the strength of your hand on this board, your position, and actions before you. Do not explain your answer.
Your optimal action is:
"""
true_action = "check"

FastLanguageModel.for_inference(model)
inputs = tokenizer(f"{sample_instruction}", return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
pred_output = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Your optimal action is:")[-1].strip()

print("=== Inference Example ===")
print(f"True Optimal Action: {true_action}")
print(f"Model Generated Action: {pred_output}")
print(f"Model name {model}")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
